# 쇼핑몰 지점별 매출액 예측 경진대회

## 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

In [2]:
sns.set_theme(style='white')

In [3]:
SEED = 22

In [4]:
os.getcwd()

'C:\\Users\\Jaesu\\github_desktop\\Dacon-Basic\\쇼핑몰지점별-매출액-예측\\쇼핑몰지점별-매출액-EDA모델링'

In [5]:
os.chdir('C:\\Users\\Jaesu\\github_desktop\\Dacon-Basic\\쇼핑몰지점별-매출액-예측')

In [6]:
df = pd.read_csv('./Data/train.csv')
test = pd.read_csv('./Data/test.csv')
submission = pd.read_csv('./Data/sample_submission.csv')

In [ ]:
df.drop('id', axis=1, inplace=True)
test.drop('id', axis=1, inplace=True)

## 데이터 전처리 및 파생변수 생성

- Data 변수를 날짜 형식으로 변경

In [43]:
def get_date(df) -> None:
    """df['Date']에서 연, 월, 일 변수를 생성""" 
    df['Date'] = pd.to_datetime(df['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day 

In [44]:
get_date(df)
get_date(test)

## 시계열 예측 모델 생성

In [15]:
import pmdarima as pm
from pmdarima.arima import ndiffs

In [110]:
def RMSE(true, pred):
    score = np.sqrt(np.mean(np.square(true - pred)))
    return score

In [45]:
date = sorted(df['Date'].unique())

In [46]:
test['Date'].unique()

array(['2012-05-10T00:00:00.000000000', '2012-12-10T00:00:00.000000000',
       '2012-10-19T00:00:00.000000000', '2012-10-26T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [47]:
train = df[['Store', 'Date', 'Weekly_Sales']]

`-` 분포가 상이한 쇼핑몰: 43, 42, 37, 30, 36(하강), 38(상승), 33

`-` 차분이 필요한지 확인하겠다

In [48]:
store_diffs = {}
for store in range(1, 46):
    y_train = train.query('Store == @store').sort_values('Date')['Weekly_Sales']
    kpss_diffs = ndiffs(y_train, test='kpss')
    adf_diffs = ndiffs(y_train, test='adf')
    n_diffs = max(adf_diffs, kpss_diffs)
    store_diffs[store] = n_diffs
    #print(f'{store} 쇼핑몰의 추정된 차수 d = {n_diffs}')

In [247]:
def fit_model(y_train, store, m=12, seasonal=True):
    model = pm.auto_arima(y=y_train       
                          ,d=store_diffs[store]            
                          ,start_p=0 
                          ,max_p=4
                          ,start_q=0 
                          ,max_q=4   
                          ,m=m
                          ,seasonal=True
                          ,stepwise=True
                          ,trace=False
                          ,random_state=SEED
                          )
    return model

In [394]:
# plt.figure(figsize=(10, 6))
# sns.lineplot(data=df.query('Store == 1'), x='Date', y='Weekly_Sales')

In [393]:
# store = 1

# y_train5 = df.query('Store == @store and Date <= "2012-04-30"').sort_values('Date')[['Date', 'Weekly_Sales']].set_index('Date') 
# y_valid5 = df.query('Store == @store and Date > "2012-04-30"').sort_values('Date')[['Date', 'Weekly_Sales']]

# y_train10 = df.query('Store == @store and Date <= "2012-09-30"').sort_values('Date')[['Date', 'Weekly_Sales']].set_index('Date') 
# y_valid10 = df.query('Store == @store and Date > "2012-09-30"').sort_values('Date')[['Date', 'Weekly_Sales']]

# y_train12 = df.query('Store == @store and Date <= "2011-11-30"').sort_values('Date')[['Date', 'Weekly_Sales']].set_index('Date') 
# y_valid12 = df.query('Store == @store and Date > "2011-11-30"').sort_values('Date')[['Date', 'Weekly_Sales']]

`-` `Prob(Q)` 값이 커야하고 `Prob(JB)` 값이 작아야 하고 `Prob(H)` 값이 커야한다

`-` test 데이터 날짜 : `2012-05-10`, `2012-10-19`, `2012-10-26`, `2012-12-10`

`-` train 데이터 날짜1 : `2012-04-27`, `2012-10-08`, `2012-10-08`, `2012-11-05`

`-` train 데이터 날짜2 : `2011-05-08`, `2011-10-14`, `2011-10-21`, `2011-12-08`

`-` 데이터가 거의 일주일 주기이므로 예측값도 일주일 주기라고 생각하면 된다

`-` train의 마지막 날짜에서 일주일씩 더하면 된다

In [427]:
def forecast_one_step(model):
    fc, conf_int = model.predict(n_periods=1, return_conf_int=True) 
    return fc.tolist()[0], np.asarray(conf_int).tolist()[0]

In [433]:
def predict_test(store, y_train5, y_valid5, y_train10, y_valid10, y_train12, y_valid12):
    model5 = fit_model(y_train5, store)
    model10 = fit_model(y_train10, store)
    model12 = fit_model(y_train12, store)
    
    ## 5월
    pred5 = model5.predict()[2] ## 2012/05/10
    
    ## 10월
    y_pred10 = []
    for new_ob in y_valid10['Weekly_Sales']:
        fc, _ = forecast_one_step(model10)
        y_pred10.append(fc)
        model10.update(new_ob)
    pred10 = [y_pred10[-1]] + model10.predict(1).tolist() ## 2012/10/19, 2012/10/6
    
    ## 12월
    y_pred12 = []
    for new_ob in y_valid12['Weekly_Sales']:
        fc, _ = forecast_one_step(model12)
        y_pred12.append(fc)
        model12.update(new_ob)
    pred12 = y_pred12[1] ## 2012/12/10
    return {'2012-05-10':pred5, '2012-10-19':pred10[0], '2012-10-26':pred10[1], '2012-12-10':pred12}

In [434]:
def predict_all_test():
    test['Weekly_Sales'] = 0
    for store in tqdm(range(1, 46)):
        ## 5월, 10월, 12월
        y_train5 = df.query('Store == @store and Date <= "2012-04-30"').sort_values('Date')[['Date', 'Weekly_Sales']].set_index('Date') 
        y_valid5 = df.query('Store == @store and Date > "2012-04-30"').sort_values('Date')[['Date', 'Weekly_Sales']]

        y_train10 = df.query('Store == @store and Date <= "2012-09-30"').sort_values('Date')[['Date', 'Weekly_Sales']].set_index('Date') 
        y_valid10 = df.query('Store == @store and Date > "2012-09-30"').sort_values('Date')[['Date', 'Weekly_Sales']]

        y_train12 = df.query('Store == @store and Date <= "2011-11-30"').sort_values('Date')[['Date', 'Weekly_Sales']].set_index('Date') 
        y_valid12 = df.query('Store == @store and Date > "2011-11-30"').sort_values('Date')[['Date', 'Weekly_Sales']]
        
        ## 예측
        test_dict = predict_test(store, y_train5, y_valid5, y_train10, y_valid10, y_train12, y_valid12)
        for d in ['2012-05-10', '2012-10-19', '2012-10-26', '2012-12-10']:    
            test.loc[(test['Store'] == store) & (test['Date'] == d), 'Weekly_Sales'] = test_dict[d]

In [439]:
## 쇼핑몰 지점별 test 예측
predict_all_test()

100%|██████████████████████████████████████████████████████████████████████████████████| 45/45 [05:36<00:00,  7.47s/it]


In [440]:
submission['Weekly_Sales'] = test['Weekly_Sales']
submission.head()

,id,Weekly_Sales
0,1,1.592866e+06
1,2,1.487614e+06
2,3,1.558230e+06
3,4,1.609455e+06
4,5,1.926600e+06


In [475]:
df.query('Store == 1 and Month==4')

,Store,Date,Temperature,Fuel_Price,Promotion1,Promotion2,Promotion3,Promotion4,Promotion5,Unemployment,IsHoliday,Weekly_Sales,Year,Month,Day
10,1,2010-04-16,66.32,2.808,NaN,NaN,NaN,NaN,NaN,7.808,False,1466058.28,2010,4,16
11,1,2010-04-23,64.84,2.795,NaN,NaN,NaN,NaN,NaN,7.808,False,1391256.12,2010,4,23
12,1,2010-04-30,67.41,2.780,NaN,NaN,NaN,NaN,NaN,7.808,False,1425100.71,2010,4,30
17,1,2010-04-06,80.69,2.705,NaN,NaN,NaN,NaN,NaN,7.808,False,1615524.71,2010,4,6
52,1,2011-04-02,42.27,2.989,NaN,NaN,NaN,NaN,NaN,7.742,False,1606629.58,2011,4,2
56,1,2011-04-03,59.58,3.288,NaN,NaN,NaN,NaN,NaN,7.742,False,1636263.41,2011,4,3
62,1,2011-04-15,71.27,3.743,NaN,NaN,NaN,NaN,NaN,7.682,False,1559889.00,2011,4,15
63,1,2011-04-22,72.99,3.807,NaN,NaN,NaN,NaN,NaN,7.682,False,1564819.81,2011,4,22
64,1,2011-04-29,72.03,3.810,NaN,NaN,NaN,NaN,NaN,7.682,False,1455090.69,2011,4,29
91,1,2011-04-11,54.98,3.332,NaN,NaN,NaN,NaN,NaN,7.866,False,1697229.58,2011,4,11


In [476]:
# x = pd.read_csv('./Data/submission16.csv') ## 제일 점수가 좋은 부스팅 모델의 예측

In [449]:
# submission.to_csv('./Data/submission_arima2.csv', index=False)

`-` 이상점을 제거하고 시도해보자 (i.g. 온도가 너무 극심하게 변하여 매출량이 매우 높아진 경우)